In [1]:
import pandas as pd
pd.set_option("max_colwidth", None)
pd.set_option("max_seq_items", None)

import numpy as np

In [2]:
path = '/Users/tristangarcia/desktop/hp-pred_data/data/wa_data/'

In [3]:
train = pd.read_csv(f'{path}wa_train.csv')
test = pd.read_csv(f'{path}wa_validation.csv')  # validation set

In [4]:
print(train.shape)
print(test.shape)

(81105, 39)
(17380, 39)


# Helper function

In [5]:
def get_group_stat(row, stat_table, groupby_col, impute_col):
    key = row[groupby_col]
    if key in stat_table.index:
        return stat_table.loc[key, impute_col]
    return None


def impute_by_group(train, test, groupby_col, impute_col, method):
    if method == 'median':
        # Compute median table
        stat_table = train.groupby(groupby_col)[impute_col].median()
    elif method == 'mode':
        # Compute mode table
        stat_table = train.groupby(groupby_col)[impute_col].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
    elif method == 'mean':
        stat_table = train.groupby(groupby_col)[impute_col].mean()
    # Setting index as the groupby column
    stat_table = stat_table.reset_index().set_index(groupby_col)
    
    # Impute missing values in train
    train[impute_col] = train.apply(
        lambda x: x[impute_col] if pd.notnull(x[impute_col]) and x[impute_col] != 0 
        else get_group_stat(x, stat_table, groupby_col, impute_col), 
        axis=1
    )
    
    # Impute missing values in test (using median/mode from the training set)
    test[impute_col] = test.apply(
        lambda x: x[impute_col] if pd.notnull(x[impute_col]) and x[impute_col] != 0 
        else get_group_stat(x, stat_table, groupby_col, impute_col), 
        axis=1
    )
    
    return train, test

# Reformatting 

### Property type

In [6]:
train['propertyTypes'].value_counts()

propertyTypes
House                                                             65920
Apartment                                                         11165
Townhouse/Villa                                                    2958
Any                                                                 784
House,Any                                                           184
House,House                                                          13
Apartment,House,Townhouse/Villa,Townhouse/Villa                      11
House,Townhouse/Villa                                                11
Apartment,House,Townhouse/Villa                                      10
House,Land                                                            8
Apartment,House                                                       7
Any,Any                                                               7
Apartment,Townhouse/Villa                                             5
Townhouse/Villa,House                             

### Only possible entries:
- House
- Apartment
- Land
- Townhouse/Villa
- Rural??
- Any??

In [7]:
train['propertyTypes'] = train['propertyTypes'].str.split(',')
test['propertyTypes'] = test['propertyTypes'].str.split(',')

def reformat_propertyTypes(row):
    # Valid property types
    types = ['House','Apartment','Townhouse/Villa']
    # This deals with NaN values
    if not isinstance(row, list):
        return 'Unknown'
    # Go through all property types
    for t in row:
        if t in types:
            # Return the first instance of a valid property type
            return t
    # Return 'Unkown' if no valid property types found
    return 'Unknown'
        
# Applying the function
train['propertyTypes'] = train['propertyTypes'].apply(reformat_propertyTypes)
test['propertyTypes'] = test['propertyTypes'].apply(reformat_propertyTypes)

In [8]:
# Changing the rural values from propertyTypes to House
train['propertyTypes'] = train['propertyTypes'].apply(lambda x: 'Townhouse' if x=='Townhouse/Villa' else x)
train.rename(columns={'propertyTypes':'propertyType'}, inplace=True)
test.rename(columns={'propertyTypes':'propertyType'}, inplace=True)

### Features

In [9]:
train['features'].value_counts()

features
['Ensuite']                                                                                                                                                                                     6397
['Air conditioning']                                                                                                                                                                            1083
['Secure Parking']                                                                                                                                                                               720
['Study']                                                                                                                                                                                        522
['Air conditioning', 'Built in wardrobes']                                                                                                                                                       327
      

In [10]:
train['features'][0][0]

'['

We can see that each instance of the variables 'features' looks like a list but is actually a string

In [11]:
# Converting the 'string lists' into lists
# https://stackoverflow.com/questions/1894269/how-to-convert-string-representation-of-list-to-a-list
import ast

train['features'] = train['features'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
test['features'] = test['features'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

### Lowercasing categorical variables

In [12]:
train = train.map(lambda x: x.lower() if isinstance(x, str) else x)
test = test.map(lambda x: x.lower() if isinstance(x,str) else x)

In [13]:
# Lowercasing features variable
def lower_features(feature_list):
    f = []
    if len(feature_list) == 0:
        return feature_list
    else:
        for feature in feature_list:
            f.append(feature.lower())
    return f

train['features'] = train['features'].apply(lambda x: lower_features(x))
test['features'] = test['features'].apply(lambda x: lower_features(x))

# Missing values

In [14]:
train.isnull().sum()

listingId                      0
unitNumber                 66800
streetNumber                 543
street                       505
suburb                        12
state                         12
postcode                      12
bathrooms                      0
bedrooms                       0
parking                        0
landArea                   15773
latitude                     543
longitude                    543
features                       0
agency                         0
propertyType                   0
promoLevel                     0
soldMonth                      0
soldYear                       0
daysListed                     0
inspectionsCount               0
isRural                        0
hasDescription                 0
hasFloorplan                   0
hasDisplayPrice                0
hasPhoto                       0
photoCount                     0
suburb_medianPrice            38
suburb_medianRentPrice        38
suburb_entryLevelPrice        38
suburb_lux

### Sold price

In [15]:
na_cols = ['suburb','soldPrice']
# Removing missing rows for soldPrice
for col in na_cols:
    train.dropna(subset=[col], inplace=True)
    test.dropna(subset=[col], inplace=True)

### Suburb insight features

In [16]:
# All rows missing rows for suburb_medianPrice,suburb_medianRentPrice,suburb_entryLevelPrice,suburb_luxuryLevelPrice
# are from the same row
train.loc[train['suburb_medianPrice'].isnull(), ['suburb','suburb_medianRentPrice','suburb_entryLevelPrice',
                                               'suburb_luxuryLevelPrice']]

,suburb,suburb_medianRentPrice,suburb_entryLevelPrice,suburb_luxuryLevelPrice
818,wandi,NaN,NaN,NaN
2457,morawa,NaN,NaN,NaN
5086,lockyer,NaN,NaN,NaN
8467,heathridge,NaN,NaN,NaN
10052,hillarys,NaN,NaN,NaN
10708,leeming,NaN,NaN,NaN
10756,castletown,NaN,NaN,NaN
14975,success,NaN,NaN,NaN
15456,ledge point,NaN,NaN,NaN
16730,broadwater,NaN,NaN,NaN


In [17]:
# Input price for the first instance 
train.loc[train["suburb"] == 'wandi',['suburb_medianPrice']].median()

suburb_medianPrice    740000.0
dtype: float64

The median house price for Winthrop in 2024 is 1,373,000
We well check train.loc[(24563)] to ensure it has been imputed with the correct value

In [18]:
test.loc[test['suburb_medianPrice'].isnull(), ['suburb','suburb_medianRentPrice','suburb_entryLevelPrice',
                                               'suburb_luxuryLevelPrice']]

,suburb,suburb_medianRentPrice,suburb_entryLevelPrice,suburb_luxuryLevelPrice
157,mundijong,NaN,NaN,NaN
5531,falcon,NaN,NaN,NaN
6030,mundijong,NaN,NaN,NaN
6119,glendalough,NaN,NaN,NaN
7887,meadow springs,NaN,NaN,NaN
9019,glendalough,NaN,NaN,NaN
10729,murdoch,NaN,NaN,NaN
11045,calista,NaN,NaN,NaN


In [19]:
# Input price for the first instance 
train.loc[train["suburb"] == 'mundijong',['suburb_medianPrice']].median()

suburb_medianPrice    0.0
dtype: float64

The median house price for balcatta in 2024 is 681000
We well check test.loc[(11432)] to ensure it has been imputed with the correct value

In [20]:
########################################################
#                                                      # 
#  Imputing missing values for the suburb features     #
#  using the median value of the corresponding suburb  #
#                                                      #
########################################################

groupby_col = 'suburb'
# Features to impute
suburb_features = ['suburb_medianPrice', 'suburb_medianRentPrice',
                   'suburb_entryLevelPrice', 'suburb_luxuryLevelPrice']

# Loop through each feature
for impute_col in suburb_features:
    train,test = impute_by_group(train, test, groupby_col, impute_col, method='median')

In [21]:
# Checking correct value has been imputed
train.loc[(818)]['suburb_medianPrice']

740000.0

In [22]:
# Checking correct value has been imputed
test.loc[(157)]['suburb_medianPrice']

0.0

In [23]:
# Removing instances where the suburb features are still 0
for feature in suburb_features:
    train = train[train[feature] > 0]
    
train.shape

(69355, 39)

### Spatial features

Using the same imputation strategy as above, we will be imputing the missing values for latitude and longitude on the train and test sets based on the suburb of the training set

In [24]:
train.loc[train['latitude'].isnull(), ['suburb','latitude','longitude']]

,suburb,latitude,longitude
18,dalkeith,NaN,NaN
69,banksia grove,NaN,NaN
148,subiaco,NaN,NaN
555,yalyalup,NaN,NaN
743,maddington,NaN,NaN
...,...,...,...
80225,bayswater,NaN,NaN
80443,morley,NaN,NaN
80603,queens park,NaN,NaN
80608,north perth,NaN,NaN


In [25]:
print(train[train['suburb']=='dalkeith']['latitude'].mean())
print(train[train['suburb']=='dalkeith']['longitude'].mean())

-31.995212007627117
115.79775083305086


In [26]:
test.loc[test['latitude'].isnull(), ['suburb','latitude','longitude']]

,suburb,latitude,longitude
586,maylands,NaN,NaN
600,bentley,NaN,NaN
804,booragoon,NaN,NaN
839,scarborough,NaN,NaN
1021,beaconsfield,NaN,NaN
...,...,...,...
16334,coodanup,NaN,NaN
16820,darkan,NaN,NaN
16952,walpole,NaN,NaN
17102,beckenham,NaN,NaN


In [27]:
print(train[train['suburb']=='maylands']['latitude'].mean())
print(train[train['suburb']=='maylands']['longitude'].mean())

-31.93234186501458
115.89808489067055


In [28]:
########################################################
#                                                      # 
#  Imputing missing values for the spatial features    #
#  using the median value of the corresponding suburb  #
#                                                      #
########################################################

groupby_col = 'suburb'
# Features to impute
spatial_features = ['latitude','longitude']

# Loop through each feature
for impute_col in spatial_features:
    train,test = impute_by_group(train, test, groupby_col, impute_col, method='mean')

In [29]:
print(train.loc[(18)]['latitude'])
print(train.loc[(18)]['longitude'])

-31.995212007627117
115.79775083305084


In [30]:
print(test.loc[(586)]['latitude'])
print(test.loc[(586)]['longitude'])

-31.93234186501458
115.89808489067055


# KNN Imputation

In [31]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic

### Land area
We will be using latitude and longitude to impute on landArea. We suspect that there is a difference in landArea for rural and non-rural areas. There also seems to be less recorded neighbours for rural areas. To account for this, the data will be split into rural and non-rural areas before imputation. 

In [32]:
# Dropping the rows with missing latitude and longitude as these columns are needed for imputation
train = train.dropna(subset=['latitude','longitude'])
test = test.dropna(subset=['latitude','longitude'])
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)
# Seperating null and non-null 
x = train.loc[train['landArea'].notnull(), ['latitude','longitude','landArea','isRural']]
y = train.loc[train['landArea'].isnull(), ['latitude','longitude','landArea','isRural']]
y_test = test.loc[test['landArea'].isnull(), ['latitude','longitude','landArea','isRural']]

In [33]:
print(x.shape)
print(y.shape)

(55079, 4)
(14276, 4)


- x has the non-null values
- y has the null values

In [34]:
y.shape

(14276, 4)

In [35]:
# Rural instances
x_rural = x.loc[x['isRural']==True, ['latitude','longitude','landArea']]
y_rural = y.loc[y['isRural']==True, ['latitude','longitude']]
y_test_rural = y_test.loc[y_test['isRural']==True, ['latitude','longitude']]
# Non-rural instances
x_nonrural = x.loc[x['isRural']==False, ['latitude','longitude','landArea']]
y_nonrural = y.loc[y['isRural']==False, ['latitude','longitude']]
y_test_nonrural = y_test.loc[y_test['isRural']==False, ['latitude','longitude']]

In [36]:
print(x_rural.shape)
print(y_rural.shape)
print(y_test_rural.shape)
print()
print(x_nonrural.shape)
print(y_nonrural.shape)
print(y_test_nonrural.shape)

(392, 3)
(0, 2)
(0, 2)

(54687, 3)
(14276, 2)
(3306, 2)


- We've now separated the rural and nonrural data
- We will using the x to predict values for y

In [37]:
# This cell is here because an older dataset version had null values for rural areas, but this new
# data set doesn't have any (so there are no values that need imputation)
'''
imputer_rural = KNeighborsRegressor(n_neighbors=1)
# Fitting on the training set
imputer_rural.fit(x_rural[['latitude','longitude']], x_rural['landArea'])
# Imputing on missing rural landArea 
predict_rural = imputer_rural.predict(y_rural)
train.loc[y_rural.index,'landArea'] = predict_rural
# Imputing on missing rural landArea on validation set using training data
predict_test_rural = imputer_rural.predict(y_test_rural)
test.loc[y_test_rural.index,'landArea'] = predict_test_rural
'''

"\nimputer_rural = KNeighborsRegressor(n_neighbors=1)\n# Fitting on the training set\nimputer_rural.fit(x_rural[['latitude','longitude']], x_rural['landArea'])\n# Imputing on missing rural landArea \npredict_rural = imputer_rural.predict(y_rural)\ntrain.loc[y_rural.index,'landArea'] = predict_rural\n# Imputing on missing rural landArea on validation set using training data\npredict_test_rural = imputer_rural.predict(y_test_rural)\ntest.loc[y_test_rural.index,'landArea'] = predict_test_rural\n"

In [38]:
imputer_nonrural = KNeighborsRegressor(n_neighbors=3)
# Fitting on the training set
imputer_nonrural.fit(x_nonrural[['latitude','longitude']], x_nonrural['landArea'])
# Imputing on missing nonrural landArea 
predict_nonrural = imputer_nonrural.predict(y_nonrural)
train.loc[y_nonrural.index, 'landArea'] = predict_nonrural
# Imputing on missing nonrural landArea on validation set using training data
predict_test_nonrural = imputer_nonrural.predict(y_test_nonrural)
test.loc[y_test_nonrural.index,'landArea'] = predict_test_nonrural

### School features
We will be using a dataset found online which includes schools and their geospatial locations. 
https://asl.acara.edu.au/school-search
The only filter I made on the search was selecting primary, secondary and combined on the type of school (leaving special unselected)

In [39]:
school_path = '/Users/tristangarcia/desktop/hp-pred_data/school/'
schools = pd.read_csv(f'{school_path}school_location.csv')

In [40]:
schools.head()

,School Name,Suburb,State,Postcode,School Sector,School Type,Latitude,Longitude
0,Corpus Christi Catholic School,BELLERIVE,TAS,7018,Catholic,Primary,-42.871256,147.371473
1,Fahan School,SANDY BAY,TAS,7005,Independent,Combined,-42.916158,147.352764
2,Geneva Christian College,LATROBE,TAS,7307,Independent,Combined,-41.226741,146.438726
3,Holy Rosary Catholic School,CLAREMONT,TAS,7011,Catholic,Primary,-42.789375,147.248306
4,Immaculate Heart of Mary Catholic School,LENAH VALLEY,TAS,7008,Catholic,Primary,-42.865543,147.290159


In [41]:
#### Preprocessing school data ####

# Dropping missing values
schools.dropna(inplace=True)
# Subsetting columns
schools = schools[['School Name', 'State','School Sector', 'School Type', 'Latitude', 'Longitude']]
# Lowercasing all string values
schools = schools.map(lambda x: x.lower() if isinstance(x,str) else x)
# Changing 'Independent' sector to 'private'
schools['School Sector'] = schools['School Sector'].map(lambda x: 'private' if x=='independent' else x)
# Renaming columns
schools.rename(columns={'School Name':'school','State':'state','School Sector':'sector',
                        'School Type':'type','Latitude':'latitude','Longitude':'longitude'}, inplace=True)

In [42]:
# Function to calculate distance in kilometers
def degree_to_km(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

In [43]:
# Function to impute school data for a given dataset (train or test) and school type
def impute_school_data(df, schools, school_type):
    # Filter the schools based on the given school type
    school_df = schools[(schools['state']=='wa')&
                     ((schools['type']==school_type) | 
                      (schools['type']=='combined'))][['school','type','sector','latitude','longitude']]

    # Initialize Nearest Neighbors model
    nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
    nn.fit(school_df[['latitude', 'longitude']])

    # Identify rows with missing values in the given dataset for the specified school type
    missing_df = df[df[f'{school_type}'].isna()]

    # Impute missing data if there are any
    if not missing_df.empty:
        distances, indices = nn.kneighbors(missing_df[['latitude', 'longitude']])

        # Calculate distances in kilometers
        distances_km = [
            degree_to_km(lat1, lon1, lat2, lon2)
            for (lat1, lon1), (lat2, lon2) in zip(
                missing_df[['latitude', 'longitude']].values,
                school_df.iloc[indices.flatten()][['latitude', 'longitude']].values
            )
        ]

        df.loc[missing_df.index, f'{school_type}'] = school_df.iloc[indices.flatten()]['school'].values
        df.loc[missing_df.index, f'{school_type}Distance'] = distances_km
        df.loc[missing_df.index, f'{school_type}Type'] = school_df.iloc[indices.flatten()]['sector'].values

    return df

In [44]:
# Imputing on training set
train = impute_school_data(train, schools, 'primary')
train = impute_school_data(train, schools, 'secondary')
# Imputing on testing set
test = impute_school_data(test, schools, 'primary')
test = impute_school_data(test, schools, 'secondary')

# Writing to file

In [45]:
train.to_pickle(f'{path}wa_train_clean.pkl')
test.to_pickle(f'{path}wa_validation_clean.pkl')